In [1]:
import ansys.fluent.core as pyfluent
from ansys.fluent.core import examples
from matplotlib import pyplot
import os, sys
import csv
import math
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
#读取参数配置文件的数据
data_all=[]
with open('data.csv','r',encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data_all.append(row)

In [3]:
#启动fluent，载入网格，检查有无负体积网格
import_filename="naca0012.cas"
solver = pyfluent.launch_fluent(version="2d",precision="double", processor_count=4,show_gui=True, mode="solver")
solver.file.read(file_type="case", file_name=import_filename)
solver.tui.mesh.check()

Multicore SMT processors detected. Processor affinity set!
Reading "naca0012.cas"...
Buffering for file scan...

   38448 nodes.
   38016 quadrilateral cells, zone  2.
   75600 2D interior faces, zone  3.
     608 2D pressure-far-field faces, zone  4.
     128 2D wall faces, zone  5.
     128 2D wall faces, zone  6.

Building...
     mesh
	auto partitioning mesh by Metis (fast),
	distributing mesh
		parts....,
		faces....,
		nodes....,
		cells....,
        bandwidth reduction using Reverse Cuthill-McKee: 9338/93 = 100.409
     materials,
     interface,
     domains,
     zones,
	wing_up
	wing_down
	far_fluent
	interior-flu
	flu
     parallel,
The automatic selection for flux type has changed to Rhie-Chow: momentum based.

Done.
  Please change either the boundary condition type or enable ideal gas law.
  Solver cannot proceed until this is fixed.
  Please change either the boundary condition type or enable ideal gas law.
  Solver cannot proceed until this is fixed.
	
 Domain Extents:


In [4]:
#定义模型：开启能量方程，sst k-omega
solver.setup.models.energy.enabled = True
#solver.tui.define.models.energy('yes')

# solver.setup.models.viscous.model='k-omega-standard'
# solver.setup.models.viscous.k_omega_model='sst'
solver.tui.define.models.viscous.kw_sst('yes')

solver.setup.models.viscous.k_omega_options.kw_low_re_correction=False
solver.setup.models.viscous.options.viscous_heating=True
#定义材料：理想气体
solver.setup.materials.fluid['air'].density.option='ideal-gas'
solver.setup.materials.fluid['air'].viscosity.option='sutherland'
#定义操作压力
solver.tui.define.operating_conditions.operating_pressure('101325')

  Please change either the boundary condition type or enable ideal gas law.
  Solver cannot proceed until this is fixed.
  Please change either the boundary condition type or enable ideal gas law.
  Solver cannot proceed until this is fixed.
  Please change either the boundary condition type or enable ideal gas law.
  Solver cannot proceed until this is fixed.


In [5]:
solver.tui.solve.report_definitions.add('bbb','lift','thread-names',
                                        'wing_up','wing_down','()','force-vector',
                                        '0.2','0.6')

In [6]:
#定义求解算法，压力速度耦合，开启全局伪时间
solver.tui.solve.set.p_v_coupling(24) # Coupled
solver.tui.solve.set.gradient_scheme('yes')    # Green-Gauss Node Based
solver.solution.methods.pseudo_time_method.formulation.coupled_solver='global-time-step'
#定义残差检测
solver.tui.solve.monitors.residual.check_convergence('yes','no','no','no','no','no')
solver.tui.solve.monitors.residual.convergence_criteria('1e-7')
solver.tui.solve.monitors.residual.monitor('yes','yes','yes','yes','yes','yes')
solver.tui.solve.monitors.residual.plot("yes")
solver.tui.solve.monitors.residual.print("yes")

	 Options:
 Green-Gauss Cell-Based 
 Green-Gauss Node-Based 
 Least-Squares Cell-Based Default 



In [7]:
#创建一个函数。目的：创建监控文件+保存参数文件+绘图
#输入参数攻角AOA
#无返回值
def create_report(need_aoa):
    aoa=float(need_aoa)
    print(aoa)
    sin_aoa=math.sin(math.radians(aoa))
    cos_aoa=math.cos(math.radians(aoa))
    #创建监控
    solver.tui.solve.report_definitions.add('lift_aoa_'+str(aoa),
                                       'lift',
                                       'force-vector',
                                       str(-1*sin_aoa),str(cos_aoa),
                                       'thread-names',
                                       'wing_down','wing_up',
                                       '()',
                                       'quit')
    
    solver.tui.solve.report_definitions.add('drag_aoa_'+str(aoa),
                                       'drag',
                                       'force-vector',
                                       str(cos_aoa),str(sin_aoa),
                                       'thread-names',
                                       'wing_down','wing_up',
                                       '()',
                                       'quit')
    
    #升力
    solver.tui.solve.report_files.add('lift_file_aoa_'+str(aoa),
                                          'report-defs',
                                          'lift_aoa_'+str(aoa),
                                          '()',
                                          'print?',
                                          'yes',
                                      'file-name',
                                      'lift_file_aoa_'+str(aoa),
                                          'quit')
    solver.tui.solve.report_plots.add('lift_plot_aoa_'+str(aoa),
                                          'report-defs',
                                          'lift_aoa_'+str(aoa),
                                          '()',
                                          'print?',
                                          'yes',
                                          'quit')
    
    #阻力
    solver.tui.solve.report_files.add('drag_file_aoa_'+str(aoa),
                                          'report-defs',
                                          'drag_aoa_'+str(aoa),
                                          '()',
                                          'print?',
                                          'yes',
                                      'file-name',
                                      'drag_file_aoa_'+str(aoa),
                                          'quit')
    
    solver.tui.solve.report_plots.add('drag_plot_aoa_'+str(aoa),
                                          'report-defs',
                                          'drag_aoa_'+str(aoa),
                                          '()',
                                          'print?',
                                          'yes',
                                          'quit')

In [8]:
#定义一个全局变量
#目的：判断是不是第一次迭代，方便之后写表头和定义一次局部流场面
target_face=0

In [9]:
#根据data数据里的case设置边界条件和迭代次数
for data in data_all:
    iter_data=data['iter_num']
    AOA_data=float(data['AOA'])
    mach_data=data['mach']
    gage_pressure_data=data['gage_pressure']
    temperature_data=data['temperature']
    wall_temperature_data=data['wall_temperature']
    print(iter_data,AOA_data,mach_data,gage_pressure_data,temperature_data,wall_temperature_data)
    sin_aoa=math.sin(math.radians(AOA_data))
    cos_aoa=math.cos(math.radians(AOA_data))
    #定义远场边界条件
    solver.setup.boundary_conditions.pressure_far_field["far_fluent"].p.constant=float(gage_pressure_data)
    solver.setup.boundary_conditions.pressure_far_field["far_fluent"].m.constant=float(mach_data)
    solver.setup.boundary_conditions.pressure_far_field["far_fluent"].t.constant=float(temperature_data)
    solver.setup.boundary_conditions.pressure_far_field["far_fluent"].flow_direction_component[0].constant=cos_aoa
    solver.setup.boundary_conditions.pressure_far_field["far_fluent"].flow_direction_component[1].constant=sin_aoa
    #定义壁面边界条件
    #这里'-1'表示绝热边界
    if wall_temperature_data=='-1':
        solver.tui.define.boundary_conditions.set.wall('wing_down,wing_up',
                                              '()',
                                              'thermal-bc','yes',
                                              'heat-flux','heat-flux',
                                              'no','0','quit')
    #定义恒温边界
    else:
        solver.tui.define.boundary_conditions.set.wall('wing_down,wing_up',
                                              '()',
                                              'thermal-bc','yes',
                                              'temperature','temperature',
                                              'no',wall_temperature_data,'quit')
    #打印查看远场边界条件
    solver.setup.boundary_conditions.pressure_far_field["far_fluent"]()
    
    #定义参考值
    solver.tui.report.reference_values.compute.pressure_far_field('far_fluent')
    solver.tui.report.reference_values.list()
    
    #创建监控文件
    create_report(AOA_data)
    
    #初始化+fmg初始化+开始迭代求解
    solver.tui.solve.initialize.compute_defaults.pressure_far_field('far_fluent')
    solver.tui.solve.initialize.initialize_flow()
    solver.tui.solve.initialize.fmg_initialization('yes')
    print('初始化完成')
    solver.solution.run_calculation.iterate(number_of_iterations=int(iter_data))
    print('迭代完成，等待输出')   
    
    
    #输出升力报告
    solver.tui.report.forces.wall_forces('yes',str(-1*sin_aoa),str(cos_aoa),'yes','naca0012_aoa_'+str(AOA_data)+'_cl.txt')
    with open('naca0012_aoa_'+str(AOA_data)+'_cl.txt','r') as f_cl:
        f_cl_line=f_cl.readlines()
        cl=list(filter(None,f_cl_line[-1].rstrip().split(' ')))[-1]
    #输出阻力报告
    solver.tui.report.forces.wall_forces('yes',str(cos_aoa),str(sin_aoa),'yes','naca0012_aoa_'+str(AOA_data)+'_cd.txt')
    with open('naca0012_aoa_'+str(AOA_data)+'_cd.txt','r') as f_cd:
        f_cd_line=f_cd.readlines()
        cd=list(filter(None,f_cd_line[-1].rstrip().split(' ')))[-1]
    cl_cd_cata={'AOA':[AOA_data],'cl':[cl],'cd':[cd]}    
    df=pd.DataFrame(cl_cd_cata)
    #第一次迭代提取局部数据面
    if target_face==0:
    #不同的模型面的interior-flu可能名字不一样
        df.to_csv('./data/cl_cd.csv',mode='a',index=False,header=True)
        solver.tui.surface.iso_clip('x-coordinate','x_-0.5_2','interior-flu','-0.5','2')
        solver.tui.surface.iso_clip('y-coordinate','y_fronx_-0.5_1','x_-0.5_2','-0.5','1')
        target_face=1
    else:    
        df.to_csv('./data/cl_cd.csv',mode='a',index=False,header=False)
    #保存局部流场数据
    solver.tui.file.export.ascii(('loc_naca0012_aoa_'+str(AOA_data)+'.txt'),'y_fronx_-0.5_1','()','no','pressure','()','no')
    solver.tui.file.write_case_data(('naca0012_aoa_'+str(AOA_data)+'.cas'))
    
    #保存完一个case之后的cas+dat后删除所有报告
    solver.tui.solve.report_definitions.delete_all('yes')
    solver.tui.solve.report_files.delete_all('yes')
    solver.tui.solve.report_plots.delete_all('yes')

50 0.0 0.15 81888.32 300 -1
 Green-Gauss Node-Based will be used(far_fluent)
        area: 1
     density: 2.1275982
    enthalpy: 3217.2117
      length: 1
    pressure: 81888.32
 temperature: 300
       yplus: 300
    velocity: 52.063734
   viscosity: 1.8461798e-05
        zone: #f
0.0
(far_fluent)

Creating multigrid levels... 
 Grid Level  0:  38016 cells,  76464 faces,  38448 nodes;   4 clusters
 Grid Level  1:   9581 cells,  38646 faces,  38447 nodes;   4 clusters
 Grid Level  1:   9581 cells,  20856 faces,      0 nodes
 Grid Level  2:   2665 cells,  21094 faces,  38445 nodes;   4 clusters
 Grid Level  2:   2665 cells,   6372 faces,      0 nodes
 Grid Level  3:    776 cells,  12117 faces,  38441 nodes;   4 clusters
 Grid Level  3:    776 cells,   2017 faces,      0 nodes
 Grid Level  4:    240 cells,   7443 faces,  38433 nodes;   4 clusters
 Grid Level  4:    240 cells,    674 faces,      0 nodes
 Grid Level  5:     79 cells,   4861 faces,  38417 nodes;   4 clusters
 Grid Level  


Writing "naca0012_aoa_0.0.dat"...

Done.

50 2.0 0.15 81888.32 300 -1
(far_fluent)
        area: 1
     density: 2.1275982
    enthalpy: 3217.2117
      length: 1
    pressure: 81888.32
 temperature: 300
       yplus: 300
    velocity: 52.063734
   viscosity: 1.8461798e-05
        zone: flu
2.0
(far_fluent)

Creating multigrid levels... 
 Grid Level  0:  38016 cells,  76464 faces,  38448 nodes;   4 clusters
 Grid Level  1:   9581 cells,  38646 faces,  38447 nodes;   4 clusters
 Grid Level  1:   9581 cells,  20856 faces,      0 nodes
 Grid Level  2:   2665 cells,  21094 faces,  38445 nodes;   4 clusters
 Grid Level  2:   2665 cells,   6372 faces,      0 nodes
 Grid Level  3:    776 cells,  12117 faces,  38441 nodes;   4 clusters
 Grid Level  3:    776 cells,   2017 faces,      0 nodes
 Grid Level  4:    240 cells,   7443 faces,  38433 nodes;   4 clusters
 Grid Level  4:    240 cells,    674 faces,      0 nodes
 Grid Level  5:     79 cells,   4861 faces,  38417 nodes;   4 clusters
 Grid


Writing "naca0012_aoa_2.0.dat"...

Done.

50 4.0 0.15 81888.32 300 -1
(far_fluent)
        area: 1
     density: 2.1275982
    enthalpy: 3217.2117
      length: 1
    pressure: 81888.32
 temperature: 300
       yplus: 300
    velocity: 52.063734
   viscosity: 1.8461798e-05
        zone: flu
4.0
(far_fluent)

Creating multigrid levels... 
 Grid Level  0:  38016 cells,  76464 faces,  38448 nodes;   4 clusters
 Grid Level  1:   9581 cells,  38646 faces,  38447 nodes;   4 clusters
 Grid Level  1:   9581 cells,  20856 faces,      0 nodes
 Grid Level  2:   2665 cells,  21094 faces,  38445 nodes;   4 clusters
 Grid Level  2:   2665 cells,   6372 faces,      0 nodes
 Grid Level  3:    776 cells,  12117 faces,  38441 nodes;   4 clusters
 Grid Level  3:    776 cells,   2017 faces,      0 nodes
 Grid Level  4:    240 cells,   7443 faces,  38433 nodes;   4 clusters
 Grid Level  4:    240 cells,    674 faces,      0 nodes
 Grid Level  5:     79 cells,   4861 faces,  38417 nodes;   4 clusters
 Grid


Writing "naca0012_aoa_4.0.dat"...

Done.

50 6.0 0.15 81888.32 300 -1
(far_fluent)
        area: 1
     density: 2.1275982
    enthalpy: 3217.2117
      length: 1
    pressure: 81888.32
 temperature: 300
       yplus: 300
    velocity: 52.063734
   viscosity: 1.8461798e-05
        zone: flu
6.0
(far_fluent)

Creating multigrid levels... 
 Grid Level  0:  38016 cells,  76464 faces,  38448 nodes;   4 clusters
 Grid Level  1:   9581 cells,  38646 faces,  38447 nodes;   4 clusters
 Grid Level  1:   9581 cells,  20856 faces,      0 nodes
 Grid Level  2:   2665 cells,  21094 faces,  38445 nodes;   4 clusters
 Grid Level  2:   2665 cells,   6372 faces,      0 nodes
 Grid Level  3:    776 cells,  12117 faces,  38441 nodes;   4 clusters
 Grid Level  3:    776 cells,   2017 faces,      0 nodes
 Grid Level  4:    240 cells,   7443 faces,  38433 nodes;   4 clusters
 Grid Level  4:    240 cells,    674 faces,      0 nodes
 Grid Level  5:     79 cells,   4861 faces,  38417 nodes;   4 clusters
 Grid


Writing "naca0012_aoa_6.0.dat"...

Done.

50 8.0 0.15 81888.32 300 -1
(far_fluent)
        area: 1
     density: 2.1275982
    enthalpy: 3217.2117
      length: 1
    pressure: 81888.32
 temperature: 300
       yplus: 300
    velocity: 52.063734
   viscosity: 1.8461798e-05
        zone: flu
8.0
(far_fluent)

Creating multigrid levels... 
 Grid Level  0:  38016 cells,  76464 faces,  38448 nodes;   4 clusters
 Grid Level  1:   9581 cells,  38646 faces,  38447 nodes;   4 clusters
 Grid Level  1:   9581 cells,  20856 faces,      0 nodes
 Grid Level  2:   2665 cells,  21094 faces,  38445 nodes;   4 clusters
 Grid Level  2:   2665 cells,   6372 faces,      0 nodes
 Grid Level  3:    776 cells,  12117 faces,  38441 nodes;   4 clusters
 Grid Level  3:    776 cells,   2017 faces,      0 nodes
 Grid Level  4:    240 cells,   7443 faces,  38433 nodes;   4 clusters
 Grid Level  4:    240 cells,    674 faces,      0 nodes
 Grid Level  5:     79 cells,   4861 faces,  38417 nodes;   4 clusters
 Grid

...


Writing "naca0012_aoa_2.0.dat"...

Done.

50 4.0 0.15 81888.32 300 300
(far_fluent)
        area: 1
     density: 2.1275982
    enthalpy: 3217.2117
      length: 1
    pressure: 81888.32
 temperature: 300
       yplus: 300
    velocity: 52.063734
   viscosity: 1.8461798e-05
        zone: flu
4.0
(far_fluent)

Creating multigrid levels... 
 Grid Level  0:  38016 cells,  76464 faces,  38448 nodes;   4 clusters
 Grid Level  1:   9581 cells,  38646 faces,  38447 nodes;   4 clusters
 Grid Level  1:   9581 cells,  20856 faces,      0 nodes
 Grid Level  2:   2665 cells,  21094 faces,  38445 nodes;   4 clusters
 Grid Level  2:   2665 cells,   6372 faces,      0 nodes
 Grid Level  3:    776 cells,  12117 faces,  38441 nodes;   4 clusters
 Grid Level  3:    776 cells,   2017 faces,      0 nodes
 Grid Level  4:    240 cells,   7443 faces,  38433 nodes;   4 clusters
 Grid Level  4:    240 cells,    674 faces,      0 nodes
 Grid Level  5:     79 cells,   4861 faces,  38417 nodes;   4 clusters
 Gri


Writing "naca0012_aoa_4.0.dat"...

Done.

50 6.0 0.15 81888.32 300 300
(far_fluent)
        area: 1
     density: 2.1275982
    enthalpy: 3217.2117
      length: 1
    pressure: 81888.32
 temperature: 300
       yplus: 300
    velocity: 52.063734
   viscosity: 1.8461798e-05
        zone: flu
6.0
(far_fluent)

Creating multigrid levels... 
 Grid Level  0:  38016 cells,  76464 faces,  38448 nodes;   4 clusters
 Grid Level  1:   9581 cells,  38646 faces,  38447 nodes;   4 clusters
 Grid Level  1:   9581 cells,  20856 faces,      0 nodes
 Grid Level  2:   2665 cells,  21094 faces,  38445 nodes;   4 clusters
 Grid Level  2:   2665 cells,   6372 faces,      0 nodes
 Grid Level  3:    776 cells,  12117 faces,  38441 nodes;   4 clusters
 Grid Level  3:    776 cells,   2017 faces,      0 nodes
 Grid Level  4:    240 cells,   7443 faces,  38433 nodes;   4 clusters
 Grid Level  4:    240 cells,    674 faces,      0 nodes
 Grid Level  5:     79 cells,   4861 faces,  38417 nodes;   4 clusters
 Gri


Writing "naca0012_aoa_6.0.dat"...

Done.

50 8.0 0.15 81888.32 300 300
(far_fluent)
        area: 1
     density: 2.1275982
    enthalpy: 3217.2117
      length: 1
    pressure: 81888.32
 temperature: 300
       yplus: 300
    velocity: 52.063734
   viscosity: 1.8461798e-05
        zone: flu
8.0
(far_fluent)

Creating multigrid levels... 
 Grid Level  0:  38016 cells,  76464 faces,  38448 nodes;   4 clusters
 Grid Level  1:   9581 cells,  38646 faces,  38447 nodes;   4 clusters
 Grid Level  1:   9581 cells,  20856 faces,      0 nodes
 Grid Level  2:   2665 cells,  21094 faces,  38445 nodes;   4 clusters
 Grid Level  2:   2665 cells,   6372 faces,      0 nodes
 Grid Level  3:    776 cells,  12117 faces,  38441 nodes;   4 clusters
 Grid Level  3:    776 cells,   2017 faces,      0 nodes
 Grid Level  4:    240 cells,   7443 faces,  38433 nodes;   4 clusters
 Grid Level  4:    240 cells,    674 faces,      0 nodes
 Grid Level  5:     79 cells,   4861 faces,  38417 nodes;   4 clusters
 Gri


Writing "naca0012_aoa_8.0.dat"...

Done.

50 10.0 0.15 81888.32 300 300
(far_fluent)
        area: 1
     density: 2.1275982
    enthalpy: 3217.2117
      length: 1
    pressure: 81888.32
 temperature: 300
       yplus: 300
    velocity: 52.063734
   viscosity: 1.8461798e-05
        zone: flu
10.0
(far_fluent)

Creating multigrid levels... 
 Grid Level  0:  38016 cells,  76464 faces,  38448 nodes;   4 clusters
 Grid Level  1:   9581 cells,  38646 faces,  38447 nodes;   4 clusters
 Grid Level  1:   9581 cells,  20856 faces,      0 nodes
 Grid Level  2:   2665 cells,  21094 faces,  38445 nodes;   4 clusters
 Grid Level  2:   2665 cells,   6372 faces,      0 nodes
 Grid Level  3:    776 cells,  12117 faces,  38441 nodes;   4 clusters
 Grid Level  3:    776 cells,   2017 faces,      0 nodes
 Grid Level  4:    240 cells,   7443 faces,  38433 nodes;   4 clusters
 Grid Level  4:    240 cells,    674 faces,      0 nodes
 Grid Level  5:     79 cells,   4861 faces,  38417 nodes;   4 clusters
 G


Writing "naca0012_aoa_10.0.dat"...

Done.



In [10]:
#退出fluent求解器
solver.exit()